In [ ]:
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

In [ ]:
class Model:
    
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def sigmoid_diff(self,x):
        return x*(1.0 - x)      # derivative of sigmoid function

    def __init__(self,num_inputs,num_hidden,num_outputs,l_rate=0.001,epochs=1000,t=0.5):
        self.num_inputs = num_inputs
        self.num_hidden = num_hidden
        self.num_outputs = num_outputs
        self.l_rate = l_rate
        self.epochs = epochs
        self.threshold = t
    
        layers = [self.num_inputs] + self.num_hidden + [self.num_outputs]


        # initializing weights, bias, activations and derivatives lists
        self.weights = []
        for i in range(len(layers)-1):
            w = np.random.randn(layers[i],layers[i+1])
            self.weights.append(w)

        self.bias = []
        for i in range(len(layers)-1):
            b = np.random.randn(1,layers[i+1])
            self.bias.append(b)
        
        self.activations = []
        for i in range(len(layers)):
            self.activations.append(np.random.randn(1,layers[i]))

        self.wderivatives = []
        for i in range(len(layers)-1):
            self.wderivatives.append(np.random.randn(layers[i],layers[i+1]))

        self.bderivatives = []
        for i in range(len(layers)-1):
            self.bderivatives.append(np.random.randn(1,layers[i+1]))

    def forward(self,inputs):
        a = inputs
        self.activations[0] = a.reshape(1,a.shape[0])
        for i,w in enumerate(self.weights):
            z = np.dot(a,w) + self.bias[i]
            a = self.sigmoid(z)
            self.activations[i+1] = a
        
        return a
    
    def backward(self,error):
        for i in reversed(range(len(self.wderivatives))):
            a_next = self.activations[i+1]
            delta = error*self.sigmoid_diff(a_next)
            delta = delta.reshape(delta.shape[0],-1)
            a_curr = self.activations[i]
            a_curr = a_curr.reshape(a_curr.shape[1],-1)
            self.wderivatives[i] = np.dot(a_curr,delta)
            self.bderivatives[i] = delta
            error = np.dot(delta,self.weights[i].T)
        
        return error

    def updating(self):
        for i in range(len(self.weights)):
            self.weights[i] += self.l_rate*self.wderivatives[i]
            self.bias[i] += self.l_rate*self.bderivatives[i]
    
    def train(self,inputs,labels):
        for i in range(self.epochs):
            for j,(x,y) in enumerate(zip(inputs,labels)):
                output = self.forward(x)
                error = y - output
                temp = self.backward(error)
                self.updating()

# Question 1

In [ ]:
train_inputs = np.random.randint(2,size=(20,7))
val_inputs = np.random.randint(2,size=(10,7))

train_outputs = []
val_outputs = []

for x in train_inputs:
    if np.sum(x,axis=0)<3:
        train_outputs.append(0)
    else:
        train_outputs.append(1)

for x in val_inputs:
    if np.sum(x,axis=0)<3:
        val_outputs.append(0)
    else:
        val_outputs.append(1)

train_outputs = np.array(train_outputs)
val_outputs = np.array(val_outputs)


In [ ]:
inst = Model(7,[15],1)
inst.train(train_inputs,train_outputs)
thres = 0.5
tot,cor = 0,0

for (inp,lab) in zip(train_inputs,train_outputs):
    out = inst.forward(inp)
    out = (out>thres)*1
    tot+=1
    if out==lab:
        cor+=1
    print(cor/tot)
tot,cor = 0,0

for (inp,lab) in zip(val_inputs,val_outputs):
    out = inst.forward(inp)
    out = (out>thres)*1
    tot+=1
    if out==lab:
        cor+=1
    print(cor/tot)

print(inst.weights)

# Question 2

In [ ]:
parent_inputs = []
parent_outputs = []

for i in range(1,100):
    temp = [int(j) for j in bin(i)[2:]]
    repr = []
    for j in range(7-len(temp)):
        repr = [0] + repr
    repr = repr+list(temp)
    parent_inputs.append(np.array(repr))
    parent_outputs.append([round(1/(i+1),3)])

parent_inputs,parent_outputs = shuffle(parent_inputs,parent_outputs,random_state=0)

train_inputs = np.array(parent_inputs[:80])
train_outputs = np.array(parent_outputs[:80])

val_inputs = np.array(parent_inputs[80:])
val_outputs = np.array(parent_outputs[80:])

In [ ]:
for hid in range(1,30):
    inst = Model(7,[hid],1)
    inst.train(train_inputs,train_outputs)

    tot,cor = 0,0

    for (inp,lab) in zip(train_inputs,train_outputs):
        out = inst.forward(inp)
        out[0][0] = round(out[0][0],2)
        lab = round(lab[0],2)
        tot+=1
        if out.item()==lab.item():
            cor+=1
    print(cor/tot)
    tot,cor = 0,0

    for (inp,lab) in zip(val_inputs,val_outputs):
        out = inst.forward(inp)
        out[0][0] = round(out[0][0],2)
        lab = round(lab[0],2)
        tot+=1
        if out.item()==lab.item():
            cor+=1
    print(cor/tot)

# Question 3

In [ ]:
train_inputs = np.loadtxt(open('train_data.csv'),delimiter=',',usecols=range(4),skiprows=(1))
train_outputs = np.loadtxt(open('train_data.csv'),delimiter=',',usecols=(4),skiprows=(1),dtype=str)
train_outputs = (train_outputs == 'versicolor')*1

train_inputs,train_outputs = shuffle(train_inputs,train_outputs,random_state=0)

test_inputs = np.loadtxt(open('test_data.csv'),delimiter=',',usecols=range(4),skiprows=(1))
test_outputs = np.loadtxt(open('test_data.csv'),delimiter=',',usecols=(4),skiprows=(1),dtype=str)
test_outputs = (test_outputs == 'versicolor')*1

test_inputs,test_outputs = shuffle(test_inputs,test_outputs,random_state=0)


inst = Model(4,[5],1)
inst.train(train_inputs,train_outputs)

thres = 0.5
tot,cor = 0,0
for (inp,lab) in zip(train_inputs,train_outputs):
    out = inst.forward(inp)
    out = (out>thres)*1
    tot+=1
    if out==lab:
        cor+=1
print(cor/tot)

tot,cor = 0,0

for (inp,lab) in zip(test_inputs,test_outputs):
    out = inst.forward(inp)
    out = (out>thres)*1
    tot+=1
    if out==lab:
        cor+=1
print(cor/tot)

